# 确定性有限自动机

In [1]:
class FARule < Struct.new(:state, :character, :next_state)
  def applies_to?(state, character)
    self.state == state && self.character == character
  end

  def follow
    next_state
  end

  def inspect
    "#<FARule #{state.inspect} --#{character}--> #{next_state.inspect}>"
  end
end


:inspect

In [2]:
class DFARulebook < Struct.new(:rules)
  def next_state(state, character)
    rule_for(state, character).follow
  end

  def rule_for(state, character)
    rules.detect { |rule| rule.applies_to?(state, character) }
  end
end

:rule_for

In [3]:
rulebook = DFARulebook.new([
  FARule.new(1,'a',2),
  FARule.new(1,'b',1),
  FARule.new(2,'a',2),
  FARule.new(2,'b',3),
  FARule.new(3,'a',3),
  FARule.new(3,'b',3),
  
  ])

#<struct DFARulebook rules=[#<FARule 1 --a--> 2>, #<FARule 1 --b--> 1>, #<FARule 2 --a--> 2>, #<FARule 2 --b--> 3>, #<FARule 3 --a--> 3>, #<FARule 3 --b--> 3>]>

In [4]:
rulebook.next_state(1,'a')

2

In [5]:
rulebook.next_state(1,"b")

1

In [6]:
rulebook.next_state(2,'b')

3

In [7]:
class DFA < Struct.new(:current_state, :accept_states, :rulebook)
  def accepting?
    accept_states.include?(current_state)
  end

  def read_character(character)
    self.current_state = rulebook.next_state(current_state, character)
  end

  def read_string(string)
    string.chars.each do |character|
      read_character(character)
    end
  end
end

:read_string

In [8]:
DFA.new(1,[1,3], rulebook).accepting?

true

In [9]:
DFA.new(1,[3], rulebook).accepting?

false

In [10]:
dfa = DFA.new(1,[3],rulebook)


#<struct DFA current_state=1, accept_states=[3], rulebook=#<struct DFARulebook rules=[#<FARule 1 --a--> 2>, #<FARule 1 --b--> 1>, #<FARule 2 --a--> 2>, #<FARule 2 --b--> 3>, #<FARule 3 --a--> 3>, #<FARule 3 --b--> 3>]>>

In [11]:
dfa.accepting?


false

In [12]:
dfa.read_character('b')


1

In [13]:
dfa.accepting?

false

In [14]:
dfa = DFA.new(1,[3], rulebook)
dfa.accepting?

false

In [15]:
dfa.read_string('baaab')
dfa.accepting?

true

In [16]:
class DFADesign < Struct.new(:start_state, :accept_states, :rulebook)
  def to_dfa
    DFA.new(start_state, accept_states, rulebook)
  end

  def accepts?(string)
    to_dfa.tap { |dfa| dfa.read_string(string) }.accepting?
  end
end


:accepts?

In [17]:
dfa_design = DFADesign.new(1,[3], rulebook)

#<struct DFADesign start_state=1, accept_states=[3], rulebook=#<struct DFARulebook rules=[#<FARule 1 --a--> 2>, #<FARule 1 --b--> 1>, #<FARule 2 --a--> 2>, #<FARule 2 --b--> 3>, #<FARule 3 --a--> 3>, #<FARule 3 --b--> 3>]>>

In [18]:
dfa_design.accepts?('a')


false

In [19]:
dfa_design.accepts?('baa')

false

In [20]:
dfa_design.accepts?('baba')

true

## 非确定性有限自动机

In [21]:
require 'set'

class NFARulebook < Struct.new(:rules)
  def next_states(states, character)
    states.flat_map { |state| follow_rules_for(state, character) }.to_set
  end

  def follow_rules_for(state, character)
    rules_for(state, character).map(&:follow)
  end

  def rules_for(state, character)
    rules.select { |rule| rule.applies_to?(state, character) }
  end
  
  # 自由移动
  def follow_free_moves(states)
    more_states = next_states(states, nil)

    if more_states.subset?(states)
      states
    else
      follow_free_moves(states + more_states)
    end
  end

  def alphabet
    rules.map(&:character).compact.uniq
  end
end

:alphabet

In [22]:
rulebook = NFARulebook.new([
  FARule.new(1,'a',1),
  FARule.new(1,'b',1),
  FARule.new(1,'b',2),
    
  FARule.new(2,'a',3),
  FARule.new(2,'b',3),
  FARule.new(3,'a',4),
  FARule.new(3,'b',4),
  ])

#<struct NFARulebook rules=[#<FARule 1 --a--> 1>, #<FARule 1 --b--> 1>, #<FARule 1 --b--> 2>, #<FARule 2 --a--> 3>, #<FARule 2 --b--> 3>, #<FARule 3 --a--> 4>, #<FARule 3 --b--> 4>]>

In [23]:
rulebook.next_states(Set[1],'b')

#<Set: {1, 2}>

In [24]:
rulebook.next_states(Set[1,2],'a')

#<Set: {1, 3}>

In [25]:
rulebook.next_states(Set[1,3],'b')

#<Set: {1, 2, 4}>

In [26]:
class NFA < Struct.new(:current_states, :accept_states, :rulebook)
  def accepting?
    (current_states & accept_states).any?
  end

  def read_character(character)
    self.current_states = rulebook.next_states(current_states, character)
  end

  def read_string(string)
    string.chars.each do |character|
      read_character(character)
    end
  end

  def current_states
    rulebook.follow_free_moves(super)
  end
end

:current_states

In [27]:
NFA.new(Set[1],[4],rulebook).accepting?

false

In [28]:
NFA.new(Set[1,2,4],[4],rulebook).accepting?

true

In [29]:
NFA.new(Set[1,2,4],[4],rulebook).accepting?

true

In [30]:
nfa = NFA.new(Set[1],[4],rulebook)

#<struct NFA current_states=#<Set: {1}>, accept_states=[4], rulebook=#<struct NFARulebook rules=[#<FARule 1 --a--> 1>, #<FARule 1 --b--> 1>, #<FARule 1 --b--> 2>, #<FARule 2 --a--> 3>, #<FARule 2 --b--> 3>, #<FARule 3 --a--> 4>, #<FARule 3 --b--> 4>]>>

In [31]:
nfa.accepting?


false

In [32]:
nfa.read_character('b')

#<Set: {1, 2}>

In [33]:
nfa.accepting?


false

In [34]:
nfa.read_character('a')

#<Set: {1, 3}>

In [35]:
nfa.accepting?

false

In [36]:
nfa.read_character('b')

#<Set: {1, 2, 4}>

In [37]:
nfa.accepting?

true

In [38]:
nfa = NFA.new(Set[1],[4],rulebook)

#<struct NFA current_states=#<Set: {1}>, accept_states=[4], rulebook=#<struct NFARulebook rules=[#<FARule 1 --a--> 1>, #<FARule 1 --b--> 1>, #<FARule 1 --b--> 2>, #<FARule 2 --a--> 3>, #<FARule 2 --b--> 3>, #<FARule 3 --a--> 4>, #<FARule 3 --b--> 4>]>>

In [39]:
nfa.accepting?


false

In [40]:
nfa.read_string('bbbbb');


["b", "b", "b", "b", "b"]

In [41]:
nfa.accepting?

true

In [42]:
class NFADesign < Struct.new(:start_state, :accept_states, :rulebook)
  def accepts?(string)
    to_nfa.tap { |nfa| nfa.read_string(string) }.accepting?
  end

  def to_nfa(current_states = Set[start_state])
    NFA.new(current_states, accept_states, rulebook)
  end
end

:to_nfa

In [43]:
nfa_design = NFADesign.new(1,[4],rulebook)

#<struct NFADesign start_state=1, accept_states=[4], rulebook=#<struct NFARulebook rules=[#<FARule 1 --a--> 1>, #<FARule 1 --b--> 1>, #<FARule 1 --b--> 2>, #<FARule 2 --a--> 3>, #<FARule 2 --b--> 3>, #<FARule 3 --a--> 4>, #<FARule 3 --b--> 4>]>>

In [44]:
nfa_design.accepts?('bab')

true

In [45]:
nfa_design.accepts?('bbbbb')

true

In [46]:
nfa_design.accepts?('bbabb')

false

## 应用正则表达式

In [47]:
module Pattern  
  def bracket(outer_precedence)
    if precedence < outer_precedence
      '(' + to_s + ')'
    else
      to_s
    end
  end

  def inspect
    "/#{self}/"
  end

  def matches?(string)
    to_nfa_design.accepts?(string)
  end
end

:matches?

In [48]:
class Empty
  include Pattern
  def to_s
    ''
  end

  def precedence
    3
  end

  def to_nfa_design
    start_state = Object.new
    accept_states = [start_state]
    rulebook = NFARulebook.new([])

    NFADesign.new(start_state, accept_states, rulebook)
  end
end

:to_nfa_design

In [49]:
class Literal < Struct.new(:character)
  include Pattern
  def to_s
    character
  end

  def precedence
    3
  end

  def to_nfa_design
    start_state = Object.new
    accept_state = Object.new
    rule = FARule.new(start_state, character, accept_state)
    rulebook = NFARulebook.new([rule])

    NFADesign.new(start_state, [accept_state], rulebook)
  end
end

:to_nfa_design

In [50]:
class Concatenate < Struct.new(:first, :second)
  include Pattern
  def to_s
    [first, second].map { |pattern| pattern.bracket(precedence) }.join
  end

  def precedence
    1
  end

  def to_nfa_design
    first_nfa_design = first.to_nfa_design
    second_nfa_design = second.to_nfa_design

    start_state = first_nfa_design.start_state
    accept_states = second_nfa_design.accept_states
    rules = first_nfa_design.rulebook.rules + second_nfa_design.rulebook.rules
    extra_rules = first_nfa_design.accept_states.map { |state|
      FARule.new(state, nil, second_nfa_design.start_state)
    }
    rulebook = NFARulebook.new(rules + extra_rules)

    NFADesign.new(start_state, accept_states, rulebook)
  end
end

:to_nfa_design

In [51]:
class Choose < Struct.new(:first, :second)
  include Pattern
  def to_s
    [first, second].map { |pattern| pattern.bracket(precedence) }.join('|')
  end

  def precedence
    0
  end

  def to_nfa_design
    first_nfa_design = first.to_nfa_design
    second_nfa_design = second.to_nfa_design

    start_state = Object.new
    accept_states = first_nfa_design.accept_states + second_nfa_design.accept_states
    rules = first_nfa_design.rulebook.rules + second_nfa_design.rulebook.rules
    extra_rules = [first_nfa_design, second_nfa_design].map { |nfa_design|
      FARule.new(start_state, nil, nfa_design.start_state)
    }
    rulebook = NFARulebook.new(rules + extra_rules)

    NFADesign.new(start_state, accept_states, rulebook)
  end
end

:to_nfa_design

In [52]:
class Repeat < Struct.new(:pattern)
  include Pattern
  def to_s
    pattern.bracket(precedence) + '*'
  end

  def precedence
    2
  end

  def to_nfa_design
    pattern_nfa_design = pattern.to_nfa_design

    start_state = Object.new
    accept_states = pattern_nfa_design.accept_states + [start_state]
    rules = pattern_nfa_design.rulebook.rules
    extra_rules =
      pattern_nfa_design.accept_states.map { |accept_state|
        FARule.new(accept_state, nil, pattern_nfa_design.start_state)
      } +
      [FARule.new(start_state, nil, pattern_nfa_design.start_state)]
    rulebook = NFARulebook.new(rules + extra_rules)

    NFADesign.new(start_state, accept_states, rulebook)
  end
end

:to_nfa_design

In [53]:
pattern =
  Repeat.new(
    Choose.new(
      Concatenate.new(Literal.new('a'), Literal.new('b')),
      Literal.new('a')
    )
  )

/(ab|a)*/

In [54]:
nfa_design = Empty.new.to_nfa_design

#<struct NFADesign start_state=#<Object:0x00000000031274a8>, accept_states=[#<Object:0x00000000031274a8>], rulebook=#<struct NFARulebook rules=[]>>

In [55]:
nfa_design.accepts?('')

true

In [56]:
nfa_design.accepts?('a')

false

In [57]:
nfa_design = Literal.new('a').to_nfa_design
nfa_design.accepts?('')
nfa_design.accepts?('a')
nfa_design.accepts?('b')

false

In [58]:
pattern.to_s

"(ab|a)*"

In [59]:
nfa_design = Empty.new.to_nfa_design

#<struct NFADesign start_state=#<Object:0x00000000031d3af0>, accept_states=[#<Object:0x00000000031d3af0>], rulebook=#<struct NFARulebook rules=[]>>

In [60]:
nfa_design.accepts?('')

true

In [61]:
nfa_design.accepts?('a')

false

In [62]:
nfa_design = Literal.new('a').to_nfa_design

#<struct NFADesign start_state=#<Object:0x00000000032a9308>, accept_states=[#<Object:0x00000000032a92e0>], rulebook=#<struct NFARulebook rules=[#<FARule #<Object:0x00000000032a9308> --a--> #<Object:0x00000000032a92e0>>]>>

In [63]:
nfa_design.accepts?('')

false

In [64]:
nfa_design.accepts?('a')

true

In [65]:
nfa_design.accepts?('b')

false

In [66]:
Empty.new.matches?('a')

false

In [67]:
pattern = Concatenate.new(Literal.new('a'), Literal.new('b'))

/ab/

In [68]:
pattern.to_s


"ab"

In [69]:
pattern.matches?('a')

false

In [70]:
pattern.matches?('ab')

true

In [71]:
pattern = Choose.new(Literal.new('a'), Literal.new('b'))

/a|b/

In [72]:
pattern.to_s


"a|b"

In [73]:
Repeat


Repeat

In [74]:
pattern =
  Repeat.new(
    Concatenate.new(
      Literal.new('a'),
      Choose.new(Empty.new, Literal.new('b'))
    )
  )

/(a(|b))*/

In [75]:
pattern.matches?('ab')

true

In [76]:
require 'treetop'

true

In [77]:
pattern = "grammar Pattern
  rule choose
    first:concatenate_or_empty '|' rest:choose {
      def to_ast
        Choose.new(first.to_ast, rest.to_ast)
      end
    }
    /
    concatenate_or_empty
  end

  rule concatenate_or_empty
    concatenate / empty
  end

  rule concatenate
    first:repeat rest:concatenate {
      def to_ast
        Concatenate.new(first.to_ast, rest.to_ast)
      end
    }
    /
    repeat
  end

  rule empty
    '' {
      def to_ast
        Empty.new
      end
    }
  end

  rule repeat
    brackets '*' {
      def to_ast
        Repeat.new(brackets.to_ast)
      end
    }
    /
    brackets
  end

  rule brackets
    '(' choose ')' {
      def to_ast
        choose.to_ast
      end
    }
    /
    literal
  end

  rule literal
    [a-z] {
      def to_ast
        Literal.new(text_value)
      end
    }
  end
end"



"grammar Pattern\n  rule choose\n    first:concatenate_or_empty '|' rest:choose {\n      def to_ast\n        Choose.new(first.to_ast, rest.to_ast)\n      end\n    }\n    /\n    concatenate_or_empty\n  end\n\n  rule concatenate_or_empty\n    concatenate / empty\n  end\n\n  rule concatenate\n    first:repeat rest:concatenate {\n      def to_ast\n        Concatenate.new(first.to_ast, rest.to_ast)\n      end\n    }\n    /\n    repeat\n  end\n\n  rule empty\n    '' {\n      def to_ast\n        Empty.new\n      end\n    }\n  end\n\n  rule repeat\n    brackets '*' {\n      def to_ast\n        Repeat.new(brackets.to_ast)\n      end\n    }\n    /\n    brackets\n  end\n\n  rule brackets\n    '(' choose ')' {\n      def to_ast\n        choose.to_ast\n      end\n    }\n    /\n    literal\n  end\n\n  rule literal\n    [a-z] {\n      def to_ast\n        Literal.new(text_value)\n      end\n    }\n  end\nend"

In [78]:
Treetop.load_from_string(pattern)

PatternParser

In [79]:
parse_tree = PatternParser.new.parse('(a(|b))*')
pattern = parse_tree.to_ast


pattern.matches?('abba')

false

In [80]:
pattern.matches?('abaab')

true

## 等价性

In [81]:

rulebook = NFARulebook.new([
  FARule.new(1, 'a', 1), FARule.new(1, 'a', 2), FARule.new(1, nil, 2),
  FARule.new(2, 'b', 3),
  FARule.new(3, 'b', 1), FARule.new(3, nil, 2)
])

#<struct NFARulebook rules=[#<FARule 1 --a--> 1>, #<FARule 1 --a--> 2>, #<FARule 1 ----> 2>, #<FARule 2 --b--> 3>, #<FARule 3 --b--> 1>, #<FARule 3 ----> 2>]>

In [82]:
nfa_design = NFADesign.new(1, [3], rulebook)

#<struct NFADesign start_state=1, accept_states=[3], rulebook=#<struct NFARulebook rules=[#<FARule 1 --a--> 1>, #<FARule 1 --a--> 2>, #<FARule 1 ----> 2>, #<FARule 2 --b--> 3>, #<FARule 3 --b--> 1>, #<FARule 3 ----> 2>]>>

In [83]:
nfa_design.to_nfa.current_states

#<Set: {1, 2}>

In [84]:
nfa_design.to_nfa(Set[2]).current_states

#<Set: {2}>

In [85]:
nfa_design.to_nfa(Set[3]).current_states

#<Set: {3, 2}>

In [86]:
nfa = nfa_design.to_nfa(Set[2, 3])

#<struct NFA current_states=#<Set: {2, 3}>, accept_states=[3], rulebook=#<struct NFARulebook rules=[#<FARule 1 --a--> 1>, #<FARule 1 --a--> 2>, #<FARule 1 ----> 2>, #<FARule 2 --b--> 3>, #<FARule 3 --b--> 1>, #<FARule 3 ----> 2>]>>

In [87]:
nfa.read_character('b'); nfa.current_states

#<Set: {3, 1, 2}>

In [88]:

class DFADesign < Struct.new(:start_state, :accept_states, :rulebook)
  def to_dfa
    DFA.new(start_state, accept_states, rulebook)
  end

  def accepts?(string)
    to_dfa.tap { |dfa| dfa.read_string(string) }.accepting?
  end
end


class NFASimulation < Struct.new(:nfa_design)
  def next_state(state, character)
    nfa_design.to_nfa(state).tap { |nfa|
      nfa.read_character(character)
    }.current_states
  end

  def rules_for(state)
    nfa_design.rulebook.alphabet.map { |character|
      FARule.new(state, character, next_state(state, character))
    }
  end

  def discover_states_and_rules(states)
    rules = states.flat_map { |state| rules_for(state) }
    more_states = rules.map(&:follow).to_set

    if more_states.subset?(states)
      [states, rules]
    else
      discover_states_and_rules(states + more_states)
    end
  end

  def to_dfa_design
    start_state   = nfa_design.to_nfa.current_states
    states, rules = discover_states_and_rules(Set[start_state])
    accept_states = states.select { |state| nfa_design.to_nfa(state).accepting? }

    DFADesign.new(start_state, accept_states, DFARulebook.new(rules))
  end
end

TypeError: superclass mismatch for class DFADesign

In [89]:
simulation = NFASimulation.new(nfa_design)
simulation.next_state(Set[1, 2], 'a')

NameError: uninitialized constant NFASimulation

In [90]:
simulation.next_state(Set[1, 2], 'b')

NoMethodError: undefined method `next_state' for nil:NilClass

In [91]:
simulation.next_state(Set[3, 2], 'b')
simulation.next_state(Set[1, 3, 2], 'b')
simulation.next_state(Set[1, 3, 2], 'a')

NoMethodError: undefined method `next_state' for nil:NilClass

In [92]:
rulebook.alphabet
simulation.rules_for(Set[1, 2])

NoMethodError: undefined method `rules_for' for nil:NilClass

In [93]:



start_state = nfa_design.to_nfa.current_states
simulation.discover_states_and_rules(Set[start_state])
nfa_design.to_nfa(Set[1, 2]).accepting?
nfa_design.to_nfa(Set[2, 3]).accepting?

dfa_design = simulation.to_dfa_design
dfa_design.accepts?('aaa')
dfa_design.accepts?('aab')
dfa_design.accepts?('bbbabb')


NoMethodError: undefined method `discover_states_and_rules' for nil:NilClass